<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Maestro_Harmonic_Formasyon_with_Zigzag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Kütüphaneler
!pip install git+https://github.com/rongardF/tvdatafeed backtesting
!pip install tradingview-screener==2.5.0
!pip installmatplotlib openpyxl matplotlib  TvDatafeed tqdm
from tvDatafeed import TvDatafeed, Interval
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Kullanıcının zaman dilimini seçmesini sağlayan fonksiyon
def get_interval_choice():
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }

    print("Lütfen zaman dilimi seçin:")
    for key, value in intervals.items():
        print(f"{key}: {value[0]}")

    choice = input("Seçiminizi yapın (1-9): ")
    return intervals.get(choice, ('4 Saat', Interval.in_4_hour))[1]

# TradingView API bağlantısı
print("TradingView oturum açılıyor...")
tv = TvDatafeed()

# BIST Hisse Senetleri
print("BIST hisse senetleri alınıyor...")
def get_bist_symbols():
    from tradingview_screener import get_all_symbols
    symbols = get_all_symbols(market='turkey')
    return sorted([symbol.replace('BIST:', '') for symbol in symbols])

hisseler = get_bist_symbols()

# Zigzag algoritması: Fiyat serisinde belirli bir yüzde sapma ile önemli dönüş noktalarını tespit eder.
def calculate_zigzag(prices, deviation=0.05):
    prices = np.array(prices)
    if len(prices) == 0:
        return []

    indices = [0]  # İlk noktayı ekle
    last_extreme_idx = 0
    last_extreme_val = prices[0]
    direction = None

    for i in range(1, len(prices)):
        change = (prices[i] - last_extreme_val) / last_extreme_val
        if direction is None:
            if abs(change) >= deviation:
                direction = "up" if change > 0 else "down"
                last_extreme_idx = i
                last_extreme_val = prices[i]
                indices.append(i)
        else:
            if direction == "up":
                if prices[i] > last_extreme_val:
                    last_extreme_idx = i
                    last_extreme_val = prices[i]
                    indices[-1] = i  # Son noktayı güncelle
                elif (last_extreme_val - prices[i]) / last_extreme_val >= deviation:
                    direction = "down"
                    last_extreme_idx = i
                    last_extreme_val = prices[i]
                    indices.append(i)
            elif direction == "down":
                if prices[i] < last_extreme_val:
                    last_extreme_idx = i
                    last_extreme_val = prices[i]
                    indices[-1] = i
                elif (prices[i] - last_extreme_val) / last_extreme_val >= deviation:
                    direction = "up"
                    last_extreme_idx = i
                    last_extreme_val = prices[i]
                    indices.append(i)
    return indices

# Harmonik formasyon tespiti: Son 5 zigzag noktasını (X, A, B, C, D) kullanarak çeşitli formasyonları kontrol eder.
def detect_harmonic_patterns(zigzag_indices, prices):
    if len(zigzag_indices) < 5:
        return None

    # Son 5 dönüş noktasını al (X, A, B, C, D)
    pts = [prices[idx] for idx in zigzag_indices[-5:]]
    X, A, B, C, D = pts

    # Fiyat farklarını hesapla
    XA = abs(A - X)
    AB = abs(B - A)
    BC = abs(C - B)
    CD = abs(D - C)

    if XA == 0 or AB == 0 or BC == 0:
        return None

    # Oran hesaplamaları
    AB_XA = AB / XA       # AB/XA oranı
    BC_AB = BC / AB       # BC/AB oranı
    CD_XA = CD / XA       # CD/XA oranı
    CD_BC = CD / BC       # CD/BC oranı

    patterns = []

    # 1. Gartley: AB ~0.618, BC [0.382, 0.886], CD ~0.786 (tolerans ±0.1)
    if 0.5 <= AB_XA <= 0.7 and 0.382 <= BC_AB <= 0.886 and 0.686 <= CD_XA <= 0.886:
        patterns.append("Gartley")

    # 2. Bat: AB [0.382, 0.5], BC [0.382, 0.886], CD [0.786, 0.986]
    if 0.382 <= AB_XA <= 0.5 and 0.382 <= BC_AB <= 0.886 and 0.786 <= CD_XA <= 0.986:
        patterns.append("Bat")

    # 3. Butterfly: AB [0.686, 0.886], BC [0.382, 0.886], CD [1.27, 1.618]
    if 0.686 <= AB_XA <= 0.886 and 0.382 <= BC_AB <= 0.886 and 1.27 <= CD_XA <= 1.618:
        patterns.append("Butterfly")

    # 4. Crab: AB [0.382, 0.618], BC [0.382, 0.886], CD [2.5, 2.8]
    if 0.382 <= AB_XA <= 0.618 and 0.382 <= BC_AB <= 0.886 and 2.5 <= CD_XA <= 2.8:
        patterns.append("Crab")

    # 5. Cypher: AB [0.382, 0.618], BC [0.382, 0.886] ve CD/BC ~1.272 (±0.1)
    if 0.382 <= AB_XA <= 0.618 and 0.382 <= BC_AB <= 0.886 and abs(CD_BC - 1.272) < 0.1:
        patterns.append("Cypher")

    # 6. AB=CD: AB ile CD neredeyse eşit ve BC/AB [0.382, 0.886]
    if (abs(AB - CD) / AB) < 0.1 and 0.382 <= BC_AB <= 0.886:
        patterns.append("AB=CD")

    # 7. Deep Crab: AB [0.382, 0.618], BC [0.382, 0.886] ve CD/BC [2.5, 3.5]
    if 0.382 <= AB_XA <= 0.618 and 0.382 <= BC_AB <= 0.886 and 2.5 <= CD_BC <= 3.5:
        patterns.append("Deep Crab")

    # 8. Double Top/Bottom: A ve C noktaları çok yakınsa (yaklaşık %3 tolerans)
    if abs(A - C) / ((A + C) / 2) < 0.03:
        patterns.append("Double Top/Bottom")

    # 9. Head and Shoulders: B, A ve C'ye göre zirve (ya da dip) konumdaysa
    if (B > A and B > C) or (B < A and B < C):
        patterns.append("Head and Shoulders")

    # 10. Symmetrical Triangle: Tüm noktalardan oluşan aralık, XA'ya göre dar ise
    if (max(pts) - min(pts)) / XA < 0.2:
        patterns.append("Symmetrical Triangle")

    # 11. Ascending/Descending Triangle: Tavan (veya taban) noktaları neredeyse eşitse
    if abs(A - C) / ((A + C) / 2) < 0.03:
        patterns.append("Ascending/Descending Triangle")

    # 12. Black/White Swan: Örnek olarak CD/XA'nın çok yüksek olması (örneğin >1.8)
    if CD_XA > 1.8:
        patterns.append("Black/White Swan")

    if patterns:
        return {"patterns": patterns, "points": pts, "indices": zigzag_indices[-5:]}
    else:
        return None

# Tarama işlemi: Seçilen zaman diliminde BIST hisse senetleri üzerinde verileri çekip formasyonları tespit ediyoruz.
results = []
selected_interval = get_interval_choice()

for symbol in hisseler:
    print(f"{symbol} için veriler çekiliyor...")
    # TvDatafeed ile verileri çekiyoruz. (exchange parametresi 'BIST' olarak belirlenmiştir.)
    df = tv.get_hist(symbol, exchange='BIST', interval=selected_interval, n_bars=200)
    if df is None or df.empty:
        print(f"{symbol} için veri bulunamadı.")
        continue

    # Zigzag dönüş noktalarını hesapla (%5'lik sapma toleransı)
    deviation_threshold = 0.05
    zz_indices = calculate_zigzag(df['close'].values, deviation=deviation_threshold)

    # Harmonik formasyon tespitini gerçekleştir
    harmonic = detect_harmonic_patterns(zz_indices, df['close'].values)
    if harmonic:
        results.append({
            "symbol": symbol,
            "patterns": harmonic["patterns"],
            "points": harmonic["points"]
        })
        print(f"{symbol}: {harmonic['patterns']} tespit edildi.")
    else:
        print(f"{symbol}: Harmonik formasyon tespit edilemedi.")

# Tespit edilen sonuçları Excel dosyasına kaydetme
if results:
    results_df = pd.DataFrame(results)
    results_df.to_excel("harmonic_patterns.xlsx", index=False)
    print("Tespit edilen formasyonlar 'harmonic_patterns.xlsx' dosyasına kaydedildi.")
else:
    print("Hiçbir formasyon tespit edilemedi.")



  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-006tmdpd
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-006tmdpd
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


Lütfen zaman dilimi seçin:
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Seçiminizi yapın (1-9): 7


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol
ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Tespit edilen formasyonlar 'harmonic_patterns.xlsx' dosyasına kaydedildi.
